In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import midynet
import seaborn as sb
import os

from itertools import product
from collections import defaultdict

## Conversion of Figure 2 data

This data has been available on narval.

In [7]:
dynamics = ["glauber", "sis", "cowan"]
acceptables = ["prior", "posterior", "evidence", "likelihood", "mutualinfo"]

all_df = {}
for d in dynamics:
    og_data = pd.read_pickle(f"data/archive/figure2/exact-{d}/recon_information.pickle")
    coupling = np.array([0.25, 0.5, 1])
    if d == "glauber":
        clabel = "coupling"
    elif d == "sis":
        clabel = "infection_prob"
    elif d == "cowan":
        clabel = "nu"
    num_steps = np.unique(np.logspace(1, 4, 100).astype("int"))
    J, T = np.meshgrid(coupling, num_steps)
    
    data = defaultdict(list)
    for i, (j, t) in enumerate(zip(J.flatten(), T.flatten())):
        data[f"data_model.{clabel}"].append(j)
        data["data_model.length"].append(t)

    for k, v in og_data[f"figure2-exact-{d}"].items():
        key, agg = k.split("-")
        if key not in acceptables:
            continue
        for i, m in enumerate(v.flatten()):
            if agg == "mid":
                data[key + "-loc"].append(m)
            elif len(data[key + "-loc"]) > len(data[key + "-scale"]):
                data[key + "-scale"].append(m)
            else:
                data[key + "-scale"][-1] = 0.5 * (data[key + "-scale"][-1] + m)
    df = pd.DataFrame(data)
    mi = midynet.statistics.Statistics.from_dataframe(df, key="mutualinfo")
    prior = midynet.statistics.Statistics.from_dataframe(df, key="prior")
    evidence = midynet.statistics.Statistics.from_dataframe(df, key="evidence")
    data["recon-loc"], data["recon-scale"] = (mi/prior).loc, (mi/prior).scale
    data["pred-loc"], data["pred-scale"] = (mi/evidence).loc, (mi/evidence).scale
    df = {f"exact-{d}":pd.DataFrame(data)}
    
    path = f"data/figure2/exact-{d}"
    os.makedirs(path, exist_ok=True)
    pd.to_pickle(df, os.path.join(path, "reconinfo.pkl"))
    

## Conversion of Figure 3 data

Original data has been generated on graham.

In [48]:


coupling = {"large": 
    {
        "exact": None,
        "meanfield": np.unique(np.concatenate([
            np.linspace(0, 0.2, 10), 
            np.linspace(0.2, 0.8, 15)
        ])),
        "annealed": np.array([0.0, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]) / 5,
    },
    "small": {
        "exact": np.linspace(0, 4, 100) / 2,
        "meanfield": np.linspace(0, 4, 20) / 2,
        "annealed": np.linspace(0, 4, 20) / 2,
    }
            
}
methods = ["exact", "meanfield", "annealed"]
sizes = ["small", "large"]
name_map = {
    "hg": "prior",
    "hx": "evidence",
    "hxg": "likelihood",
    "hgx": "posterior",
    "mi": "mutualinfo",
}
path = "data/archive/figure3/"
for m, s in product(methods, sizes):
    data = defaultdict(list)    
    if coupling[s].get(m) is None:
        continue
    data["data_model.coupling"] = coupling[s].get(m)
    if s == "small":
        name = f"figure3-small-ising.{m}"
    else:
        name = f"figure3-large-T1000-ising.{m}"
    og_data = pd.read_pickle(f"{path}/{name}/mutualinfo.pickle")
    for k, v in og_data[name].items():
        key, agg = k.split("-")
        key = name_map[key]
        if agg == "mid":
            data[key + "-loc"] = v
        elif len(data[key + "-loc"]) > len(data[key + "-scale"]):
            data[key + "-scale"] = v
        else:
            data[key + "-scale"] = 0.5 * (data[key + "-scale"] + v)
    df = pd.DataFrame(data)
    mi = midynet.statistics.Statistics.from_dataframe(df, key="mutualinfo")
    prior = midynet.statistics.Statistics.from_dataframe(df, key="prior")
    evidence = midynet.statistics.Statistics.from_dataframe(df, key="evidence")
    data["recon-loc"], data["recon-scale"] = (mi/prior).loc, (mi/prior).scale
    data["pred-loc"], data["pred-scale"] = (mi/evidence).loc, (mi/evidence).scale
    df = pd.DataFrame(data)
    p = f"data/figure3/glauber-{s}-{m}"
    os.makedirs(p, exist_ok=True)
    pd.to_pickle({f"glauber-{s}-{m}": df}, os.path.join(p, "reconinfo.pkl"))
    
    

In [47]:
ls data/figure3

glauber-large-annealed/  glauber-small-annealed/  glauber-small-meanfield/
glauber-large-meanfield/ glauber-small-exact/


## Conversion of Figure 4 data
Original data has been generated narval.

In [121]:
og_data = pd.read_pickle("data/archive/figure4/figure4-large-nbinom-ising/mutualinfo.pickle")
dynamics = ["glauber", "sis", "cowanfw", "cowanbw"]

dmap = {
    "glauber": "ising" ,
    "sis": "sis" ,
    "cowanfw": "cowan_forward" ,
    "cowanbw": "cowan_backward" ,
}
name_map = {
    "hg": "prior",
    "hx": "evidence",
    "hxg": "likelihood",
    "hgx": "posterior",
    "mi": "mutualinfo",
}
clabel = {
    "glauber": "coupling",
    "sis": "infection_prob",
    "cowanfw": "nu",
    "cowanbw": "nu",
}

coupling = {
    "glauber": np.concatenate([np.linspace(0., 0.5, 6), np.linspace(0.5, 0.75, 22), np.linspace(0.75, 1.5, 6)])/5,
    "sis": np.concatenate([np.linspace(0., 0.125, 3), np.linspace(0.125, 0.5, 30)])/5,
    "cowanfw": np.concatenate([np.linspace(2., 2.5, 23),np.linspace(2.5, 4, 10),])/5,
    "cowanbw": np.concatenate([np.linspace(1.0, 1.25, 4),np.linspace(1.25, 1.5, 20), np.linspace(1.5, 3, 10)])/5,
    
}

for d in dynamics:
    data = defaultdict(list)
    data[f"data_model.{clabel[d]}"] = np.unique(coupling[d])
    og_data = pd.read_pickle(f"data/archive/figure4/figure4-large-nbinom-{dmap[d]}/mutualinfo.pickle")
    for k, v in og_data[f"figure4-large-nbinom-{dmap[d]}"].items():
        key, agg = k.split("-")
        key = name_map[key]
        if agg == "mid":
            data[key + "-loc"] = v
        elif len(data[key + "-loc"]) > len(data[key + "-scale"]):
            data[key + "-scale"] = v
        else:
            data[key + "-scale"] = 0.5 * (data[key + "-scale"] + v)
    df = pd.DataFrame(data)
    mi = midynet.statistics.Statistics.from_dataframe(df, key="mutualinfo")
    prior = midynet.statistics.Statistics.from_dataframe(df, key="prior")
    evidence = midynet.statistics.Statistics.from_dataframe(df, key="evidence")
    data["recon-loc"], data["recon-scale"] = (mi/prior).loc, (mi/prior).scale
    data["pred-loc"], data["pred-scale"] = (mi/evidence).loc, (mi/evidence).scale
    df = pd.DataFrame(data)
    p = f"data/figure4/geometric-{d}"
    os.makedirs(p, exist_ok=True)
    pd.to_pickle({f"geometric-{d}": df}, os.path.join(p, "reconinfo.pkl"))
        